# Summary

Crawl a bunch of sites and save data to BQ

In [1]:
import sys, os

import pandas as pd
import numpy as np

# Asynchronous work
import asyncio

from google.oauth2 import service_account
import pandas_gbq

# Web crawling and scraping tools class
sys.path.insert(0, "../../scrapers/")
import web_scraper as ws
import web_crawler as wc


## Test the site scraper on one site

In [2]:
turl = "https://en.wikipedia.org/wiki/Chaffey_College"
turl = "https://asdjhfbads.jhbf"
turl = "https://www.cccco.edu/"

test = await ws.webScraper.visit_page(url=turl)

test.crawl_results.keys()
# type(test.crawl_results)


dict_keys(['url', 'html_code_string', 'soup', 'ptag_text', 'atag_urls'])

## Web Crawler

## Set up GCP credentials

In [ ]:
# Set GCP credentials
creds_path = "../../../data/environment/"
creds_file = "eternal-bongo-435614-b9-bf6a5e630e44.json"
credentials = service_account.Credentials.from_service_account_file(filename=os.path.join(creds_path,
                                                                                          creds_file))

# Set credentials
pandas_gbq.context.credentials = credentials


## Get a list of previously crawled websites

In [ ]:
# Query the BigQuery crawl results table to get a list of sites already crawled
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
project_id = "eternal-bongo-435614-b9"

# Create a sql statement
sql = ("SELECT DISTINCT url FROM `{}` ").format(table_id)
sql = ("SELECT * FROM `{}` ").format(table_id)

# Use Pandas-gbq to read from the BQ table
# df_pc = pandas_gbq.read_gbq(sql, project_id=project_id)

########### Get a list of previously crawled sites
########### This needs some work - don't crawl may not work since we still need the URLs
# crawled_urls = df_pc["url"].unique().tolist()

# dont_crawl_urls = []

# len(crawled_urls)

## Crawl websites (and write results to OneDrive)

In [8]:
# Crawl seed_url and web links found on it and its child pages
# seed_url = "https://www.cccco.edu"
seed_url = "https://en.wikipedia.org/wiki/California_Community_Colleges"
seed_url = "https://www.ccleague.org"
seed_url = "https://cccaoe.org/"

dont_crawl_urls = []

crawler = wc.webCrawler(seed_url=seed_url)
crlres = await crawler.crawl_sites(dont_crawl_urls=dont_crawl_urls,
                                   depth=10,
                                   width=300)


 10%|█         | 1/10 [00:14<02:09, 14.35s/it]

Depth level finished: 1: 1 URLs crawled; 5 URLs in to_crawl_urls; 2 URLs in dont_crawl_urls


 20%|██        | 2/10 [01:14<05:28, 41.01s/it]

Depth level finished: 2: 6 URLs crawled; 17 URLs in to_crawl_urls; 12 URLs in dont_crawl_urls


 30%|███       | 3/10 [04:22<12:37, 108.24s/it]

Depth level finished: 3: 23 URLs crawled; 37 URLs in to_crawl_urls; 46 URLs in dont_crawl_urls


 40%|████      | 4/10 [06:57<12:41, 126.90s/it]

Depth level finished: 4: 60 URLs crawled; 1 URLs in to_crawl_urls; 120 URLs in dont_crawl_urls


100%|██████████| 10/10 [06:59<00:00, 41.93s/it]

Depth level finished: 5: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Depth level finished: 6: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Depth level finished: 7: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Depth level finished: 8: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Depth level finished: 9: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Depth level finished: 10: 61 URLs crawled; 0 URLs in to_crawl_urls; 122 URLs in dont_crawl_urls
Batch 1 saved to disk


## Crawl pages ## Crawl websites (and write results to OneDrive)

In [3]:
seed_urls= ["https://lao.ca.gov/Publications/Report/4531", 
            "https://lao.ca.gov/Publications/Report/4853",
            "https://lao.ca.gov/Publications/Report/4664",
            "https://lao.ca.gov/Publications/Report/4913"
            ]

dont_crawl_urls = []

for seed_url in seed_urls:
    
    # Set a base name (otherwise these can write over each other)
    output_filename_base = "laocagov_pub{}".format(seed_url[-4:])
    
    # Instantiate a crawler object
    crawler = wc.webCrawler(seed_url=seed_url,
                            output_filename_base = output_filename_base)

    # Crawl the page
    crlres = await crawler.crawl_sites(dont_crawl_urls=dont_crawl_urls,
                                       depth=1,
                                       width=1)



100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


Depth level finished: 1: 1 URLs crawled; 33 URLs in to_crawl_urls; 2 URLs in dont_crawl_urls
Batch 1 saved to disk


100%|██████████| 1/1 [00:02<00:00,  2.49s/it]


Depth level finished: 1: 1 URLs crawled; 33 URLs in to_crawl_urls; 4 URLs in dont_crawl_urls
Batch 1 saved to disk


100%|██████████| 1/1 [00:02<00:00,  2.49s/it]


Depth level finished: 1: 1 URLs crawled; 32 URLs in to_crawl_urls; 6 URLs in dont_crawl_urls
Batch 1 saved to disk


100%|██████████| 1/1 [00:02<00:00,  2.51s/it]

Depth level finished: 1: 1 URLs crawled; 32 URLs in to_crawl_urls; 8 URLs in dont_crawl_urls
Batch 1 saved to disk


## Read local crawl results

In [5]:
crawl_data_path  = ("/Users/stephengodfrey/OneDrive - numanticsolutions.com"
                    "/Engagements/Projects/ccc_policy_assistant/data/crawls")
crawl_file = "enwikipediaorg_2024Dec11_10.csv"
crawl_file = 'laocagov_2024Dec17_1.csv'
crawl_file = "wwwccleagueorg_2024Dec17_1.csv"

os.listdir(crawl_data_path)

dft = pd.read_csv(filepath_or_buffer=os.path.join(crawl_data_path,crawl_file))


In [7]:
dft.loc[0,'seed_url']
dft.loc[0,'ptag_text']

dft


,Unnamed: 0,seed_url,url,html_code_string,ptag_text,atag_urls,crawl_time
0,0,https://www.ccleague.org,https://www.ccleague.org,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/district-services/b...,2024-12-17 15:01:30
1,1,https://www.ccleague.org,https://www.ccleague.org/district-services/lib...,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:34
2,2,https://www.ccleague.org,https://www.ccleague.org/district-services/fin...,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:37
3,3,https://www.ccleague.org,https://www.ccleague.org/trustees/,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:39
4,4,https://www.ccleague.org,https://www.ccleague.org/district-services/sta...,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:41
5,5,https://www.ccleague.org,https://www.ccleague.org/district-services/stu...,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:45
6,6,https://www.ccleague.org,https://www.ccleague.org/advocacy/bill-tracking/,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:49
7,7,https://www.ccleague.org,https://www.ccleague.org/district-services/ene...,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:52
8,8,https://www.ccleague.org,https://www.ccleague.org/ceos/,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...",The Community College League of California is...,['https://www.ccleague.org/resources#research-...,2024-12-17 15:01:56
9,9,https://www.ccleague.org,https://www.ccleague.org/about-us/contact-us/,"<!DOCTYPE html>\n<html class=""no-js"" lang=""en-...","If you have any questions about the League, d...",['https://www.ccleague.org/resources#research-...,2024-12-17 15:02:00


## Save results to BigQuery

In [ ]:
# Identify the table; note the database.table naming schema
table_id = "ccc_polasst.crawl_data"
table_id = "ccc_polasst.crawl_data_tes"
project_id = "eternal-bongo-435614-b9"

# Use Pandas-gbq to write to the BQ table
if_exists = "replace"
# if_exists = "append"

# pandas_gbq.to_gbq(dataframe=df, destination_table=table_id,
#                   project_id=project_id, if_exists=if_exists)

